In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import seaborn as sns
from datetime import datetime
from google.colab import files
import gpt_2_simple as gpt2

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
!nvidia-smi

Sun Mar 29 19:05:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 255Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 114Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 540Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:06, 79.8Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 446Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 146Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 229Mit/s]                                                       


using the 124M model because collabatory has some issues with larger models

## Mounting Google Drive


In [0]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import json

code below takes facebook style `message_[0-9].json` messages downloaded from fb archive and translates them to gpt2 compatible txt files

In [0]:
df = pd.DataFrame()
for i in range(1, 13):
    f = open("./message_{}.json".format(i))
    messages = json.load(f)
    df = df.append(pd.DataFrame(messages['messages']))

In [0]:
txt =df.apply(lambda x: "{}: {}".format(x['sender_name'],x['content']), axis=1)
#txt = txt.unique()
type(txt)

In [0]:
txt = pd.Series(txt.unique())


In [0]:
import numpy as np

In [0]:
np.savetxt(r'./messages.txt', txt.values, fmt="%s")

In [0]:
file_name = "messages.txt"

# gpt2 tuning

- `restore_from`: `fresh` for new, `latest` for last checkpoint
    - !!!!!!!for some reason restoring from checkpoints cannot be used with a new tensorflow session???
- `sample_every` -> generates a sample every n steps
- `print_every` -> steps to print
- `learning_rate` 
    - Actually needs to be tuned for different hyperparams, 1e-4 is the default, but check https://github.com/minimaxir/gpt-2-simple#differences-between-gpt-2-simple-and-other-text-generation-utilities
    


In [0]:
sess = gpt2.start_tf_sess()


In [0]:

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='latest',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=100,
              overwrite=True
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:05<00:00,  5.37s/it]


dataset has 1170687 tokens
Training...
Saving checkpoint/run1/model-1000
Saving checkpoint/run1/model-1000
======== SAMPLE 1 ========

Dang FuFu: what do u mean
Peixian Wang: like if you don't think you can swim it's a badge of admission lol
Peixian Wang: if they don't make enough money they don't get slapped on a registry lol
Dang FuFu: but i have to live off of ramen for the next month :^)
Dang FuFu: lol
Dang FuFu: u eat like cat shit
Peixian Wang: yeah eating isn't the same as sleeping easy :^)
Dang FuFu: u also definitely have to be lazy to take care of yourself :^)
Dang FuFu: its luniks
Peixian Wang: every day is like 1/4 of mardu
Dang FuFu: i work a 10hr day
Peixian Wang: yeah people are fukn busy but I feel I've outgrown all of p3k Australia
Dang FuFu: :^)
Dang FuFu: are u going to do ur $100 dinner at work?
Dang FuFu: id really dont want to
Peixian Wang: if id spend money at work I feel like I've earned it
Dang FuFu: lol
Peixian Wang: and I've definitely wasted $100 on sushi in

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


In [0]:
gpt2.generate(sess, run_name='run1')

John Ryan Hamilton: Itâs actually kinda nuts
John Ryan Hamilton: I have like 5 other decks that canât get into Tasigur
Peixian Wang: And also seems less likely to go to mythic for cycling lands
Peixian Wang: I mean it seems like you're gonna get busted if you do get into the mythic
John Ryan Hamilton: Lol
Peixian Wang: It's like a bad unplayable card
John Ryan Hamilton: Not in a bad order
Peixian Wang: It's like a bad unplayable card
John Ryan Hamilton: Youâll be worried about it
Peixian Wang: It's an uncommon
John Ryan Hamilton: Itâs not gonna get busted
Peixian Wang: Maybe itâll get trashed as part of the traverse?
John Ryan Hamilton: Okay so we have another 0 board clear before we get into the second half of the turn
Peixian Wang: And also does not seem very good in the mirror
Peixian Wang: Yeah trample and walker are also 0 clears
John Ryan Hamilton: A completely new game plan
John Ryan Hamilton: Also like walker and trident
John Ryan Hamilton: I think walker is way bette

In [0]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="mana",
              nsamples=5,
              batch_size=5
              )

mana
John Ryan Hamilton: And it's not like the GP is a yearly thing
John Ryan Hamilton: With the exception of the fact that it's not a yearly tournament
John Ryan Hamilton: Well you can technically bring your friend
Peixian Wang: So it's like not *quite* GP
Peixian Wang: There's no requirements for entry
Peixian Wang: No you can come to the tournament *by* you can come to the cash bar at the venue
John Ryan Hamilton: Cash is accepted at the venue
John Ryan Hamilton: No tcg? No, not yet
Peixian Wang: Oh wait you can come to the cash bar at the venue?
John Ryan Hamilton: Cash is accepted at the venue
Peixian Wang: Extra fees*
John Ryan Hamilton: So how much does it cost to bring a cosplaying llr? 500? 1000?
Peixian Wang: It's free?
John Ryan Hamilton: Also if you're going you wanna come to the cash bar at the venue
John Ryan Hamilton: https://twitter.com/faristart/status/11976474498261144
Peixian Wang: https://twitter.
mana
John Ryan Hamilton: Where are the timetables
John Ryan Hamilton:

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )